In [2]:
import pandas as pd
from scipy.optimize import curve_fit

In [3]:
tablePath= "/home/marcosperrude/Documents/LCQAR/BRAVES/evaporativas_posto"
temperatura_mensal = [25, 25, 24, 22, 19, 17, 16, 17, 19, 21, 22, 24, 21]

##Importação das planilhas
sheet_volume = pd.read_excel(tablePath + "/SIC 48003009498202458.xlsx", skiprows=4, sheet_name="2000") #Litros de gasolina movimentados em uma cidade, por mês
voc_density = pd.read_csv(tablePath + "/VOC_density.csv") ## REVER UNIDADE PLANILHA VOC DENSITY (FOI CONSIDERADO g/ml, a correta está anotada no código do Braves)

In [14]:
voc_density

,temp_C,VOC_gaso_dens,VOC_eth_dens
0,-10,0.30640,0.65280
1,-9,0.30612,0.65208
2,-8,0.30584,0.65136
3,-7,0.30556,0.65064
4,-6,0.30528,0.64992
...,...,...,...
56,46,0.29072,0.61248
57,47,0.29044,0.61176
58,48,0.29016,0.61104
59,49,0.28988,0.61032


## Fatores de Emissão

In [4]:
def func(x, a, b, c, d):
    return a*x**3 + b*x**2 +c*x + d

###NAO considera loop de temperatura
##Função para o cálculo do fator de emissão do abastecimento do carro (car refueling)
#def carRefuelingEF(tamb,tablePath,ethanolPercentageGasolineC, ethanolPercentageAEHC):
    tConv = (tamb - 273.15)*((9/5)) + 32 # Kelvin para Farenheit
    # ISSO DEPENDE DO TEOR DE ALCOOL
    rvpCurve = pd.read_csv(tablePath+'/RVP.csv')
    popt, pcov = curve_fit(func, rvpCurve['ETHANOL'], rvpCurve['RVP'])
    rvpGasolineC = func(ethanolPercentageGasolineC, *popt)
    rvpAEHC= func(ethanolPercentageAEHC, *popt) 
    #Td = Dispensed gasoline temperature (degF) = 20.30+0.81*Tamb  
    #from MOVES https://www.epa.gov/sites/default/files/2020-11/documents/420r20012.pdf

    td = 20.30+0.81*tConv 

    # From MOVES - #https://www.epa.gov/sites/default/files/2020-11/documents/420r20012.pdf
    deltaT = 0.418*td -16.6   
    
    EFCarRefuelingGasolineC = 264.2*(-5.909 - 0.0949*deltaT + 0.084*td + 0.485*rvpGasolineC)
    EFCarRefuelingAEHC = 264.2*(-5.909 - 0.0949*deltaT + 0.084*td + 0.485*rvpAEHC)
    
    #rvpEthanol = 2 # 100% de etanol
    #efCarRefuelingETHANOL = 264.2*(-5.909 - 0.0949*deltaT + 0.084*td + 0.485*rvpEthanol) ##Talvez não seja necessário pois o etanol é comercializado como AEHC (possui teor de água)
    
    return EFCarRefuelingGasolineC, EFCarRefuelingAEHC

In [5]:
popt, pcov = curve_fit(func, rvpCurve['ETHANOL'], rvpCurve['RVP'])
popt

NameError: name 'rvpCurve' is not defined

In [21]:
pcov

array([[ 2.42925209e-11, -3.60615326e-09,  1.32483552e-07,
        -7.26251239e-07],
       [-3.60615326e-09,  5.53773277e-07, -2.14495632e-05,
         1.27697745e-04],
       [ 1.32483552e-07, -2.14495632e-05,  9.06434544e-04,
        -6.36144263e-03],
       [-7.26251239e-07,  1.27697745e-04, -6.36144263e-03,
         7.68764106e-02]])

In [7]:
def func(x, a, b, c, d):
    return a*x**3 + b*x**2 +c*x + d

# Função atualizada para lidar com listas de temperatura
def carRefuelingEF(tamb_list, tablePath, ethanolPercentageGasolineC, ethanolPercentageAEHC):
    
    # Carregar a curva RVP
    rvpCurve = pd.read_csv(tablePath + '/RVP.csv')
    popt, pcov = curve_fit(func, rvpCurve['ETHANOL'], rvpCurve['RVP'])
    
    # Inicializar listas para armazenar os resultados
    EFCarRefuelingGasolineC_list = []
    EFCarRefuelingAEHC_list = []

    # Loop para calcular os fatores de emissão mês a mês
    for tamb in tamb_list:
        
        tamb_K = tamb + 273.15
        # Converter temperatura de Kelvin para Fahrenheit
        tConv = (tamb_K - 273.15) * (9/5) + 32

        # Calcular RVP para cada produto
        rvpGasolineC = func(ethanolPercentageGasolineC, *popt)
        rvpAEHC = func(ethanolPercentageAEHC, *popt)

        # Calcular Td e deltaT
        td = 20.30 + 0.81 * tConv
        deltaT = 0.418 * td - 16.6

        # Calcular fatores de emissão
        EFCarRefuelingGasolineC = 264.2 * (-5.909 - 0.0949 * deltaT + 0.084 * td + 0.485 * rvpGasolineC)
        EFCarRefuelingAEHC = 264.2 * (-5.909 - 0.0949 * deltaT + 0.084 * td + 0.485 * rvpAEHC)

        # Adicionar os valores às listas
        EFCarRefuelingGasolineC_list.append(EFCarRefuelingGasolineC)
        EFCarRefuelingAEHC_list.append(EFCarRefuelingAEHC)

    return EFCarRefuelingGasolineC_list, EFCarRefuelingAEHC_list

    
tablePath= "/home/marcosperrude/Documents/LCQAR/BRAVES/evaporativas_posto"

#Obtendo EF para os serviços considerados
tamb_list = temperatura_mensal
EFCarRefuelingGasolineC_list, EFCarRefuelingAEHC_list = carRefuelingEF(tamb_list, tablePath, 27, 93)

In [26]:
EFCarRefuelingGasolineC_list

[np.float64(1070.1384899969112),
 np.float64(1070.1384899969112),
 np.float64(1053.0617210424314),
 np.float64(1018.9081831334715),
 np.float64(967.6778762700311),
 np.float64(933.5243383610713),
 np.float64(916.4475694065912),
 np.float64(933.5243383610713),
 np.float64(967.6778762700311),
 np.float64(1001.8314141789913),
 np.float64(1018.9081831334715),
 np.float64(1053.0617210424314),
 np.float64(1001.8314141789913)]

In [10]:
##Função para o cálculo de EF de acordo com a voltatilidade do produto (RVP), quando EF é dado apenas em função do serviço, calculado proporcionalmente a volatilidade da gasolina E10, dos EUA
def rvp(tablePath,ethanolPercentage, gasolineEmissionServiceEF):
    rvpCurve = pd.read_csv(tablePath+'/RVP.csv')
    popt, pcov = curve_fit(func, rvpCurve['ETHANOL'], rvpCurve['RVP'])
    rvp = func(ethanolPercentage, *popt)
    
    #Emissão proporcional ao RVP
    rvpUsaGasoline = 9.965801227 #rvp do combustivel nos EUA, considerando 10% de etanol (E10 mais comum)
    EF_produto = gasolineEmissionServiceEF *(rvp/rvpUsaGasoline) #Fazendo o fator de emissão do serviço retirado da AP42 ser proporcional ao rvp do composto
    return EF_produto


EFSubmergedFilling_GasolC = rvp(tablePath,27,880)
EFSubmergedFilling_AEHC =  rvp(tablePath,93,880)

EFTankBreathing_GasolC = rvp(tablePath,27,120)
EFTankBreathing_AEHC= rvp(tablePath,93,120)

In [15]:
EFTankBreathing_GasolC

np.float64(117.165303665793)

In [14]:
EFSubmergedFilling_GasolC

np.float64(859.212226882482)

## Emissões

In [16]:
##Função para o calculo das emissões do Car Refueling
def CarRefueling(EFCarRefuelingGasolineC_list, EFCarRefuelingAEHC_list, sheet_volume, temperatura_mensal):
    ##Parte 1: obter os volumes manipulados (Lthroughput)
    sheet_volume.columns = sheet_volume.columns.astype(str)
    colunas_meses = [str(col) for col in sheet_volume.columns if str(col).startswith('20')]
    colunas_meses = [col for col in colunas_meses if col in sheet_volume.columns]

    # Filtrar volumes por cidade e produto
    volume_gasolC = sheet_volume[
        (sheet_volume['NOM_LOCALIDADE_D'] == 'FLORIANOPOLIS') & 
        (sheet_volume['NOM_GRUPO_PRODUTO'] == 'GASOLINA C')][colunas_meses].values.flatten().tolist()
    
    volume_AEHC = sheet_volume[
        (sheet_volume['NOM_LOCALIDADE_D'] == 'FLORIANOPOLIS') & 
        (sheet_volume['NOM_GRUPO_PRODUTO'] == 'AEHC')][colunas_meses].values.flatten().tolist()

    ##Parte 2: cálculo das emissões mensais separadas
    emissoes_mensais_gasolC = [volume * EF for volume, EF in zip(volume_gasolC, EFCarRefuelingGasolineC_list)]
    emissoes_mensais_AEHC = [volume * EF for volume, EF in zip(volume_AEHC, EFCarRefuelingAEHC_list)]

    ##Parte 3: densidade correspondente à temperatura mensal
    voc_gaso_density = [
        voc_density.loc[voc_density['temp_C'] == temp, 'VOC_gaso_dens'].values[0]
        for temp in temperatura_mensal]
    
    # Converter de g/mL para mg/L (multiplicação por 10^6)
    voc_gaso_density = [value * 1_000_000 for value in voc_gaso_density]

    ##Parte 4: Cálculo do volume de VOC evaporado mensalmente (Emissões [mgVOC] / roVOC [mg/L])
    vol_evaporado_gasolC_CarRefueling = [
        emissao / dens for emissao, dens in zip(emissoes_mensais_gasolC, voc_gaso_density)]
    
    vol_evaporado_AEHC_CarRefueling = [
        emissao / dens for emissao, dens in zip(emissoes_mensais_AEHC, voc_gaso_density)]

    ## Retornar volumes e emissões separadas para Gasolina C e AEHC
    return volume_gasolC, volume_AEHC, vol_evaporado_gasolC_CarRefueling, vol_evaporado_AEHC_CarRefueling

In [17]:
# Chamada da função Car Refueling e unpaking dos retornos
volume_gasolC, volume_AEHC, vol_evaporado_gasolC_CarRefueling, vol_evaporado_AEHC_CarRefueling = CarRefueling(EFCarRefuelingGasolineC_list, EFCarRefuelingAEHC_list, sheet_volume, temperatura_mensal)
print(vol_evaporado_gasolC_CarRefueling, vol_evaporado_AEHC_CarRefueling)

[np.float64(45.95335593702347), np.float64(41.029852958433494), np.float64(39.39776741007927), np.float64(33.05259151925813), np.float64(30.388517671031227), np.float64(33.81167425228404), np.float64(31.879887315948128), np.float64(30.0077906128038), np.float64(31.689802397819562), np.float64(35.87027227959108), np.float64(36.61207312721686), np.float64(45.23514094113541)] [np.float64(1.1878746385735532), np.float64(1.113650225037117), np.float64(1.034544784585915), np.float64(0.9606181644177364), np.float64(0.5763484485573906), np.float64(0.4154526575663201), np.float64(0.4011563033585107), np.float64(0.42119435273505507), np.float64(0.7712743428515824), np.float64(0.6054515767656752), np.float64(0.7933451044409526), np.float64(1.0296527717671666)]


In [12]:
tamb_list

[25, 25, 24, 22, 19, 17, 16, 17, 19, 21, 22, 24, 21]

In [18]:
##Função para o cálculo das emissões do Submerged Filling
def SubmergedFilling(EFSubmergedFilling_GasolC, EFSubmergedFilling_AEHC, sheet_volume, temperatura_mensal):
    ##Parte 1: obter os volumes manipulados (Lthroughput)
    sheet_volume.columns = sheet_volume.columns.astype(str)
    colunas_meses = [str(col) for col in sheet_volume.columns if str(col).startswith('20')]
    colunas_meses = [col for col in colunas_meses if col in sheet_volume.columns]

    # Filtrar volumes por cidade e produto
    volume_gasolC = sheet_volume[
        (sheet_volume['NOM_LOCALIDADE_D'] == 'FLORIANOPOLIS') & 
        (sheet_volume['NOM_GRUPO_PRODUTO'] == 'GASOLINA C')][colunas_meses].values.flatten().tolist()
    
    volume_AEHC = sheet_volume[
        (sheet_volume['NOM_LOCALIDADE_D'] == 'FLORIANOPOLIS') & 
        (sheet_volume['NOM_GRUPO_PRODUTO'] == 'AEHC')][colunas_meses].values.flatten().tolist()

    ##Parte 2: cálculo das emissões mensais separadas
    emissoes_mensais_gasolC = [volume * EFSubmergedFilling_GasolC for volume in volume_gasolC]
    emissoes_mensais_AEHC = [volume * EFSubmergedFilling_AEHC for volume in volume_AEHC]

    ##Parte 3: densidade correspondente à temperatura mensal
    voc_gaso_density = [
        voc_density.loc[voc_density['temp_C'] == temp, 'VOC_gaso_dens'].values[0]
        for temp in temperatura_mensal]
    
    # Converter de g/mL para mg/L (multiplicação por 10^6)
    voc_gaso_density = [value * 1_000_000 for value in voc_gaso_density]

    ##Parte 4: Cálculo do volume de VOC evaporado mensalmente (Emissões [mgVOC] / roVOC [mg/L])
    vol_evaporado_gasolC_SubmergedFilling = [
        emissao / dens for emissao, dens in zip(emissoes_mensais_gasolC, voc_gaso_density)]
    
    vol_evaporado_AEHC_SubmergedFilling = [
        emissao / dens for emissao, dens in zip(emissoes_mensais_AEHC, voc_gaso_density)]

    ## Retornar volumes e emissões separadas para Gasolina C e AEHC
    return volume_gasolC, volume_AEHC, vol_evaporado_gasolC_SubmergedFilling, vol_evaporado_AEHC_SubmergedFilling

In [19]:
# Chamada da função Submerged Filling e unpaking dos retornos
volume_gasolC, volume_AEHC, vol_evaporado_gasolC_SubmergedFilling, vol_evaporado_AEHC_SubmergedFilling = SubmergedFilling(EFSubmergedFilling_GasolC, EFSubmergedFilling_AEHC, sheet_volume, temperatura_mensal)
print(vol_evaporado_gasolC_SubmergedFilling, vol_evaporado_AEHC_SubmergedFilling)

[np.float64(36.89586502723327), np.float64(32.942793534300584), np.float64(32.145355580015725), np.float64(27.872178507941847), np.float64(26.982311552298352), np.float64(31.120135528478816), np.float64(29.888877321410437), np.float64(27.619055590469184), np.float64(28.13773710798189), np.float64(30.763835200242536), np.float64(30.873773911284317), np.float64(36.908174900614235)] [np.float64(1.3336542879326236), np.float64(1.250320824814812), np.float64(1.236717080004541), np.float64(1.3191826735704404), np.float64(1.0188374204152284), np.float64(0.9083728759134633), np.float64(0.9949495421152557), np.float64(0.9209268939418824), np.float64(1.363416807783962), np.float64(0.8982622981525707), np.float64(1.0894725445617577), np.float64(1.230869062694233)]


In [20]:
##Função para o cálculo das emissões do Submerged Filling
def TankBreathing(EFTankBreathing_GasolC, EFTankBreathing_AEHC, sheet_volume, temperatura_mensal):
    ##Parte 1: obter os volumes manipulados (Lthroughput)
    sheet_volume.columns = sheet_volume.columns.astype(str)
    colunas_meses = [str(col) for col in sheet_volume.columns if str(col).startswith('20')]
    colunas_meses = [col for col in colunas_meses if col in sheet_volume.columns]

    # Filtrar volumes por cidade e produto
    volume_gasolC = sheet_volume[
        (sheet_volume['NOM_LOCALIDADE_D'] == 'FLORIANOPOLIS') & 
        (sheet_volume['NOM_GRUPO_PRODUTO'] == 'GASOLINA C')][colunas_meses].values.flatten().tolist()
    
    volume_AEHC = sheet_volume[
        (sheet_volume['NOM_LOCALIDADE_D'] == 'FLORIANOPOLIS') & 
        (sheet_volume['NOM_GRUPO_PRODUTO'] == 'AEHC')][colunas_meses].values.flatten().tolist()

    ##Parte 2: cálculo das emissões mensais separadas
    emissoes_mensais_gasolC = [volume * EFTankBreathing_GasolC for volume in volume_gasolC]
    emissoes_mensais_AEHC = [volume * EFTankBreathing_AEHC for volume in volume_AEHC]

    ##Parte 3: densidade correspondente à temperatura mensal
    voc_gaso_density = [
        voc_density.loc[voc_density['temp_C'] == temp, 'VOC_gaso_dens'].values[0]
        for temp in temperatura_mensal]
    
    # Converter de g/mL para mg/L (multiplicação por 10^6)
    voc_gaso_density = [value * 1_000_000 for value in voc_gaso_density]

    ##Parte 4: Cálculo do volume de VOC evaporado mensalmente (Emissões [mgVOC] / roVOC [mg/L])
    vol_evaporado_gasolC_TankBreathing = [
        emissao / dens for emissao, dens in zip(emissoes_mensais_gasolC, voc_gaso_density)]
    
    vol_evaporado_AEHC_TankBreathing = [
        emissao / dens for emissao, dens in zip(emissoes_mensais_AEHC, voc_gaso_density)]

    ## Retornar volumes e emissões separadas para Gasolina C e AEHC
    return volume_gasolC, volume_AEHC, vol_evaporado_gasolC_TankBreathing, vol_evaporado_AEHC_TankBreathing

In [21]:
# Chamada da função Tank Breathing e unpaking dos retornos 
volume_gasolC, volume_AEHC, vol_evaporado_gasolC_TankBreathing, vol_evaporado_AEHC_TankBreathing = TankBreathing(EFTankBreathing_GasolC, EFTankBreathing_AEHC, sheet_volume, temperatura_mensal)
print(vol_evaporado_gasolC_TankBreathing, vol_evaporado_AEHC_TankBreathing)

[np.float64(5.031254321895446), np.float64(4.4921991183137155), np.float64(4.383457579093054), np.float64(3.8007516147193425), np.float64(3.6794061207679576), np.float64(4.243654844792566), np.float64(4.0757559983741505), np.float64(3.766234853245798), np.float64(3.836964151088439), np.float64(4.19506843639671), np.float64(4.210060078811498), np.float64(5.03293294099285)] [np.float64(0.18186194835444866), np.float64(0.17049829429292895), np.float64(0.16864323818243743), np.float64(0.17988854639596916), np.float64(0.13893237551116752), np.float64(0.12386902853365409), np.float64(0.13567493756117122), np.float64(0.12558094008298396), np.float64(0.1859204737887221), np.float64(0.12249031338444147), np.float64(0.14856443789478516), np.float64(0.16784578127648633)]


## Volume evaporado total 

In [22]:
VolEvaporadoVOC_ProvenienteGasolC = [sum(values) for values in zip(vol_evaporado_gasolC_CarRefueling, 
                                  vol_evaporado_gasolC_SubmergedFilling, 
                                  vol_evaporado_gasolC_TankBreathing)]

print(VolEvaporadoVOC_ProvenienteGasolC)

[np.float64(87.88047528615219), np.float64(78.46484561104779), np.float64(75.92658056918805), np.float64(64.72552164191931), np.float64(61.05023534409754), np.float64(69.17546462555543), np.float64(65.84452063573272), np.float64(61.39308105651878), np.float64(63.66450365688989), np.float64(70.82917591623033), np.float64(71.69590711731267), np.float64(87.1762487827425)]


In [24]:
VolEvaporadoVOC_ProvenienteAEHC = [sum(values) for values in zip(vol_evaporado_AEHC_CarRefueling, 
                                  vol_evaporado_AEHC_SubmergedFilling, 
                                  vol_evaporado_AEHC_TankBreathing)]

print(VolEvaporadoVOC_ProvenienteAEHC)

[np.float64(2.7033908748606255), np.float64(2.534469344144858), np.float64(2.4399051027728937), np.float64(2.459689384384146), np.float64(1.7341182444837866), np.float64(1.4476945620134374), np.float64(1.5317807830349377), np.float64(1.4677021867599214), np.float64(2.320611624424266), np.float64(1.6262041883026874), np.float64(2.0313820868974957), np.float64(2.4283676157378857)]


In [25]:
VolEvaporadoVOC_Total = [sum(values) for values in zip(VolEvaporadoVOC_ProvenienteGasolC, 
                        VolEvaporadoVOC_ProvenienteAEHC)]  #Em L/mês

print(VolEvaporadoVOC_Total)

[np.float64(90.58386616101282), np.float64(80.99931495519265), np.float64(78.36648567196094), np.float64(67.18521102630346), np.float64(62.784353588581325), np.float64(70.62315918756887), np.float64(67.37630141876765), np.float64(62.8607832432787), np.float64(65.98511528131415), np.float64(72.45538010453302), np.float64(73.72728920421017), np.float64(89.60461639848039)]
